In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd

BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.Ticker as Ticker
import method.SaleRules as _sr
import method.algofuncs as _af
path = os.getcwd()

Loading BokehJS ...

In [2]:
class FollowTheTrend(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        opens = self.data.Open
        volumes = self.data.Volume
        lows = pd.DataFrame(data=self.data.Low).Low
        highs = pd.DataFrame(data=self.data.High).High
        if len(prices) > 23:
            last_price = prices[-1]
            atr_22 = _sr.getATR(22, highs, lows)
            if self.buy_price == 0 and Ticker.isFollowTrendingV2(prices, volumes, opens[-1], 2.5):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and ((_sr.takeProfitByPercent(5, last_price, self.buy_price) and opens[-1] < prices[-1] and abs(highs.iloc[-1] - lows.iloc[-1]) > atr_22) or _sr.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [3]:
ticker_id = 'DAH'
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
ticker = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2018-01-01', '2020-10-05')
bt = Backtest(ticker, FollowTheTrend, commission=.005, exclusive_orders=False)
stats = bt.run()

In [5]:
stats

Start                     2018-01-02 00:00:00
End                       2020-10-05 00:00:00
Duration                   1007 days 00:00:00
Exposure Time [%]                     37.5546
Equity Final [$]                      19842.2
Equity Peak [$]                       19842.2
Return [%]                            98.4219
Buy & Hold Return [%]                 17.6604
Return (Ann.) [%]                     28.5759
Volatility (Ann.) [%]                 24.4471
Sharpe Ratio                          1.16889
Sortino Ratio                         2.91658
Calmar Ratio                          1.79135
Max. Drawdown [%]                    -15.9522
Avg. Drawdown [%]                    -3.82558
Max. Drawdown Duration      182 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   11
Win Rate [%]                          90.9091
Best Trade [%]                        15.8003
Worst Trade [%]                      -12.5064
Avg. Trade [%]                    

In [6]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2015,103,123,4.960378,5.2436,570.691323,0.057097,2018-06-08,2018-07-06,28 days
1,1903,124,204,5.552022,5.8700,605.112134,0.057272,2018-07-09,2018-10-30,113 days
2,1811,241,294,6.170700,6.8000,1139.662300,0.101982,2018-12-20,2019-03-14,84 days
3,1571,304,305,7.839000,8.1800,535.711000,0.043500,2019-03-28,2019-03-29,1 days
4,1536,321,322,8.361600,8.7800,642.662400,0.050038,2019-04-23,2019-04-24,1 days
5,1398,446,453,9.648000,10.7000,1470.696000,0.109038,2019-10-21,2019-10-30,9 days
6,1567,473,490,9.547500,10.5000,1492.567500,0.099764,2019-11-27,2019-12-20,23 days
7,1882,510,552,8.743500,7.6500,-2057.967000,-0.125064,2020-01-20,2020-03-25,65 days
8,1557,555,558,9.246000,10.0000,1173.978000,0.081549,2020-03-30,2020-04-03,4 days
9,1335,596,605,11.658000,13.5000,2459.070000,0.158003,2020-05-29,2020-06-11,13 days


In [11]:
bt.plot()